# Connexion à la base de données

```

pip install firebase-admin

```

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate('Files/devia-f6704-firebase-adminsdk-ay7ci-26941e7999.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()

---
---

# Traitement des données

### Convertir les données en json

In [ ]:
import json

with open ("Files/donnees1.txt", "r") as file:
    datas = file.read().split("\n")

with open ("Files/donnees2.txt", "r") as file:
    datas.extend(file.read().split("\n"))

final_data = {}
for i, data in enumerate(datas):
    final_data[i] = json.loads(data)

with open ("Files/donnees.json", "w") as file:
    json.dump(final_data, file)

---
---

# Ajout des données à Firestore


### Importer les données

In [ ]:
import os, json

with open ("Files/donnees.json", "r") as file:
    datas = json.load(file)

---

### Upload des données sur la BDD Firebase

In [ ]:
for key in datas: # Notre Json contient tous les users dans un dictionnaire avec les clefs numérotés ("0", "1", ...)
    db.collection('users').document(datas[key]["user"]).set(datas[key]["personalData"]) # On ajoute le champ "personnalData" au document <userID> (3ac88a6d-bbcf-4ab2-88f2-9df2e1dd32b0 par ex)

    for tags_key in list(datas[key]["tags"].keys()): # On récupere tous les noms de tags

        db.collection('users').document(datas[key]["user"]).collection("tags").document(tags_key).set({"algo" : datas[key]["tags"][tags_key]['algo']}) # Pour chaque user, pour chaque tag, on ajoute un document au nom du tag avec un champ "algo" : "A_".

        for element in datas[key]["tags"][tags_key]["hist"]: # On récupère la liste "hist" pour chaque user dans notre fichier json, on itère les éléments
            
            # Pour chaque élément on réorganise le timestamp et l'url dans 1 seul dico
            dict_to_add = {}
            dict_to_add["ts"] =  element["ts"]["$date"]
            dict_to_add["url"] = element["url"]

            db.collection('users').document(datas[key]["user"]).collection("tags").document(tags_key).collection('hist').add(dict_to_add) # On ajoute ce dico dans la collection 'hist' dans un document (Id généré aléatoirement).

---
---

# Résultat

![alternative text](Files/Screenshot_BDD_1.png)
![alternative text](Files/Screenshot_BDD_2.png)
![alternative text](Files/Screenshot_BDD_3.png)